# Capstone - IBM Applied Data Science #
# Segmenting, Clustering,GeoCoding, Mapping Neighborhoods#

This notebook is the first step of peer-graded project evaluating neighborhoods in a Canadian province.

Processes completed in this notebook include:    

1.  Download data via Internet page scrape,        
2.  Using the data from the page scrape, create pandas dataframe,     
3.  Clean data in pandas dataframe (remove/rename empty cells),     
4.  Explore and cluster neighborhoods,
5.  Geocode using postal codes,    
6.  GeoMap neighborhoods,
7.  Assign FourSquare feature data to neighborhoods,
8.  Cluster neighborhoods based on FourSquare-assigned features,
9.  Group, One-Hot Encode and Analyze Neighborhood Clusters,
10. Visualize Feature-Grouped Clusters,
11. Examine Feature-Grouped Clusters.

### Step 0 - Libraries and Notebook Setup

In [1]:
# library imports
import pandas as pd
pd.set_option('display.max_columns', None)    
pd.set_option('display.max_rows', None)
import numpy as np
from pandas.io.json import json_normalize # JSON file -> pandas DF
import json # JSON file manipulation
import requests # HTTP library
from bs4 import BeautifulSoup # scrapes info from web pages, on top of HTML or XLM parser
import matplotlib.pyplot as plt # plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors
# %matplotlib inline # magic that allows matplot in Jupyter
# !conda update -n base -c defaults conda --yes     # may needto run first time in new environment
# !conda install -c conda-forge folium=0.5.0 --yes  # may need to run first time in new environment
import folium # map rendering
from sklearn.cluster import KMeans # clustering algorithm

/home/lin/anaconda3/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


Sklearn library note: Large pink FutureWarning re sklearn.cluster library Kmeans -- currently when comparing two arrays of dtype=object, numpy checks if the return of the comparison function is False when both objects being compared are the exact same. Right now (2019), it assumes that all objects compare equal to themselves, but that will be changed at same time in the future.  Since it's a Future Warning, ignoring for now. (this info courtesy of stackoverflow, https://stackoverflow.com/questions/43591503/error-when-using-numpy-to-encode-categorical-features-of-dataset

### Step 1 - Download data via Internet page scrape ###
Download Canadian post code data from wikipedia (url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")        
and scrape using BeautifulSoup 

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url).text  # wikiArticle -> text document
wikiHtml = BeautifulSoup(page, "html.parser")  # text -> parseTree
neighborhoodTable = wikiHtml.find("table", class_ = "wikitable")  # exracting info from html data
neighborhoodRows = neighborhoodTable.find_all("tr")  # exracting info from html data

### Step 2 - Create and Populate Pandas Dataframe ###
Project instructions require:
- Dataframe has three columns: PostalCode, Borough, and Neighborhood.
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- Group on postal code; more than one neighborhood can exist in one postal code area. 
- If cell has a borough but a Not assigned neighborhood, then neighborhood = borough.

In [3]:
# create and set up dataframe
neighborhoodInfo = []
colNames = ['PostalCode','Borough','Neighborhood'] # project requires columns PostalCode, Borough, Neighborhood
for row in neighborhoodRows:
    hoodInfo = row.text.split('\n')[1:-1]
    neighborhoodInfo.append(hoodInfo)
hoodDF = pd.DataFrame(neighborhoodInfo[1:], columns = colNames)
initialDataFrameShape = hoodDF.shape
print("initial dataframe shape", initialDataFrameShape)
print(hoodDF.head())

initial dataframe shape (289, 3)
  PostalCode           Borough      Neighborhood
0        M1A      Not assigned      Not assigned
1        M2A      Not assigned      Not assigned
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront


### Step 3 - Clean data in pandas dataframe (remove/rename empty cells)

In [4]:
# remove entries with notAssigned Borough
noBorough = hoodDF.index[hoodDF["Borough"] == "Not assigned"] 
hoodDF.drop(hoodDF.index[noBorough], inplace=True)
hoodDF.reset_index(drop=True, inplace=True)
print("after removing cells without assigned boroughs, dataframe shape:", hoodDF.shape)
hoodDF.head()

after removing cells without assigned boroughs, dataframe shape: (212, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [5]:
# rename neighborhood=borough for cells with a borough but a Not assigned neighborhood
noHood = hoodDF.index[hoodDF["Neighborhood"] == "Not assigned"] 
for hoodIndex in noHood:
    hoodDF["Neighborhood"][hoodIndex] = hoodDF["Borough"][hoodIndex]
hoodDF.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### Step 4 - Explore and Cluster Neighborhoods

In [6]:
postalCodeGroup = hoodDF.groupby('PostalCode')
hoodGroups = postalCodeGroup['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
boroughGroups = postalCodeGroup['Borough'].apply(lambda x: set(x).pop())
neighborhood = pd.DataFrame(list(zip(boroughGroups.index, boroughGroups, hoodGroups)))
neighborhood.columns = ['PostalCode', 'Borough', 'Neighborhood']
print("The dataframe when clustered by postal codes is", neighborhood.shape, '\n')
print(neighborhood.head(10))

The dataframe when clustered by postal codes is (103, 3) 

  PostalCode      Borough                                     Neighborhood
0        M1B  Scarborough                                   Rouge, Malvern
1        M1C  Scarborough           Highland Creek, Rouge Hill, Port Union
2        M1E  Scarborough                Guildwood, Morningside, West Hill
3        M1G  Scarborough                                           Woburn
4        M1H  Scarborough                                        Cedarbrae
5        M1J  Scarborough                              Scarborough Village
6        M1K  Scarborough      East Birchmount Park, Ionview, Kennedy Park
7        M1L  Scarborough                  Clairlea, Golden Mile, Oakridge
8        M1M  Scarborough  Cliffcrest, Cliffside, Scarborough Village West
9        M1N  Scarborough                      Birch Cliff, Cliffside West


In [7]:
# print(neighborhood.tail(25))

In [8]:
# borough_count = len(neighborhood['Borough'].unique())
# neighborhood_count = neighborhood.shape[0]

# print("Dataframe has", borough_count, "boroughs and", neighborhood_count, "neighborhoods")

### Step 5 - Geocode

In [9]:
# create DF from csv file w geographical coordinates of each postal code ("https://cocl.us/Geospatial_data")
geocodeDF = pd.read_csv("https://cocl.us/Geospatial_data") 
geocodedHood = neighborhood.merge(geocodeDF, left_on='PostalCode', right_on='Postal Code') # csv file uses 'Postal Code'
geocodedHood = geocodedHood.drop('Postal Code', 1)   # drop extraneous heading, leaving only 'PostalCode'
print(geocodedHood.head())

  PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Rouge, Malvern  43.806686   
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  


### Step 6 - GeoMap neighborhoods

In [10]:
# get the long/lat coordinates of Toronto, Canada

#!conda update -n base conda  --yes # may need to uncomment this the first time run
#!conda install -c conda-forge geopy --yes # may need to uncomment this the first time run
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
toronto_latitude = location.latitude
toronto_longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(toronto_latitude, toronto_longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [11]:
# create map of Toronto using latitude and longitude values
Toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(geocodedHood['Latitude'], geocodedHood['Longitude'], geocodedHood['Borough'], geocodedHood['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='pink',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto)  
    
Toronto

### Step 7 - Assign FourSquare Feature Data to Neighborhoods

In [12]:
# REMOVE THIS SECTION PRIOR TO POSTING ON GITHUB #
CLIENT_ID = 'need to use your own client id'
CLIENT_SECRET = 'need to use your own client secret'
VERSION = '20180605'
LIMIT = 100  # limit of number of venues returned by Foursquare API
radius = 3833# search radius - started at 500 & kept upping it
# REMOVE THIS SECTION PRIOR TO POSTING ON GITHUB ... or at least mask out CLIENT_ID and CLIENT_SECRET #

In [13]:
# neighborhood.loc[0, 'Neighborhood']
neighborhood_latitude = geocodedHood.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = geocodedHood.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = geocodedHood.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


In [14]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
print('url=', url)# display URL
results = requests.get(url).json()
results

url= https://api.foursquare.com/v2/venues/explore?&client_id=VUZ4KT3YVGDRUTVIB4FASBWWCA1TGR2E1N5VJRKA2ZXLYJPU&client_secret=AY3TK344OPA3VLDKTJUZVKVZGJCTVPIHU30EER3BFXNMRRTG&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=3833&limit=100


{'meta': {'code': 200, 'requestId': '5c58d5c19fb6b7681c9447eb'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4cd854fd3ec4b1f71900be3f-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/zoo_',
          'suffix': '.png'},
         'id': '58daa1558bbb0b01f18ec1fd',
         'name': 'Zoo Exhibit',
         'pluralName': 'Zoo Exhibits',
         'primary': True,
         'shortName': 'Zoo Exhibit'}],
       'id': '4cd854fd3ec4b1f71900be3f',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 1509,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.81772505914066,
          'lng': -79.18343284457424}],
        'lat': 43.81772505914

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# code lifted from cognitive.class.ai labs

### Step 8 - Cluster Neighborhoods based on FourSquare Assigned Features

In [16]:
#clean json results and Xform to pandas DF
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']  # filter columns
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)  # filter the category for each row
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]  # clean columns
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()
# code lifted from cognitive.class.ai labs

72 venues were returned by Foursquare.


,name,categories,lat,lng
0,African Rainforest Pavilion,Zoo Exhibit,43.817725,-79.183433
1,Toronto Pan Am Sports Centre,Athletics & Sports,43.790623,-79.193869
2,Toronto Zoo,Zoo,43.820582,-79.181551
3,Polar Bear Exhibit,Zoo,43.823372,-79.185145
4,Canadiana exhibit,Zoo Exhibit,43.817962,-79.193374


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
# code lifted from labs.cognitiveclass.ai/tools

In [18]:
ontario_venues = getNearbyVenues(names=geocodedHood['Neighborhood'],
                                   latitudes=geocodedHood['Latitude'],
                                   longitudes=geocodedHood['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

In [19]:
# Examine the ontario_venues dataset
print('ontario venues size', ontario_venues.shape)
print('There are {} uniques categories.'.format(len(ontario_venues['Venue Category'].unique())))
print('\nVenues returned for each neighborhood', ontario_venues.groupby('Neighborhood').count())

ontario venues size (2237, 7)
There are 277 uniques categories.

Venues returned for each neighborhood                                                     Neighborhood Latitude  \
Neighborhood                                                                
Adelaide, King, Richmond                                              100   
Agincourt                                                               4   
Agincourt North, L'Amoreaux East, Milliken, Ste...                      3   
Albion Gardens, Beaumond Heights, Humbergate, J...                     12   
Alderwood, Long Branch                                                 10   
Bathurst Manor, Downsview North, Wilson Heights                        18   
Bayview Village                                                         4   
Bedford Park, Lawrence Manor East                                      25   
Berczy Park                                                            55   
Birch Cliff, Cliffside West                       

### Step 9 - Analyze, One-Hot Encode and Group Neighborhoods

In [20]:
# Analyze each neighborhood and one-hot encode features
ontario_onehot = pd.get_dummies(ontario_venues[['Venue Category']], prefix="", prefix_sep="")
ontario_onehot['Neighborhood'] = ontario_venues['Neighborhood'] 
fixed_columns = [ontario_onehot.columns[-1]] + list(ontario_onehot.columns[:-1]) # neighborhood column as first column
ontario_onehot = ontario_onehot[fixed_columns]
print('ontario_onehot dataframe size', ontario_onehot.shape)
ontario_onehot.head()

ontario_onehot dataframe size (2237, 277)


,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [21]:
# group by neighborhood
ontario_grouped = ontario_onehot.groupby('Neighborhood').mean().reset_index()
ontario_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0

In [22]:
num_top_venues = 5

for hood in ontario_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ontario_grouped[ontario_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04


----Agincourt----
                venue  freq
0      Sandwich Place  0.25
1              Lounge  0.25
2      Breakfast Spot  0.25
3  Chinese Restaurant  0.25
4         Yoga Studio  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0         Coffee Shop  0.33
1          Playground  0.33
2                Park  0.33
3         Yoga Studio  0.00
4  Miscellaneous Shop  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.17
1  Japanese Restaurant  0.08
2          Video Store  0.08
3  Fried Chicken Joint  0.08
4       Sandwich Place  0.08


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  

                      venue  freq
0             Moving Target  0.25
1             Grocery Store  0.25
2                      Bank  0.25
3             Shopping Mall  0.25
4  Mediterranean Restaurant  0.00


----East Birchmount Park, Ionview, Kennedy Park----
              venue  freq
0    Discount Store  0.25
1       Coffee Shop  0.12
2       Bus Station  0.12
3  Department Store  0.12
4     Train Station  0.12


----East Toronto----
               venue  freq
0       Intersection   0.2
1        Coffee Shop   0.2
2               Park   0.2
3      Metro Station   0.2
4  Convenience Store   0.2


----Emery, Humberlea----
                             venue  freq
0                   Baseball Field   1.0
1        Middle Eastern Restaurant   0.0
2                            Motel   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.15
1  Fast Food Restaurant  0.08

                       venue  freq
0                Coffee Shop  0.09
1             Clothing Store  0.09
2                       Café  0.04
3  Middle Eastern Restaurant  0.03
4             Cosmetics Shop  0.03


----Scarborough Village----
                       venue  freq
0              Women's Store   0.5
1                 Playground   0.5
2  Middle Eastern Restaurant   0.0
3                      Motel   0.0
4        Monument / Landmark   0.0


----Silver Hills, York Mills----
               venue  freq
0          Cafeteria   1.0
1  Mobile Phone Shop   0.0
2      Moving Target   0.0
3      Movie Theater   0.0
4              Motel   0.0


----St. James Town----
            venue  freq
0     Coffee Shop  0.07
1      Restaurant  0.06
2           Hotel  0.05
3            Café  0.05
4  Clothing Store  0.04


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0         Coffee Shop  0.10
1          Restaurant  0.05
2                Café  0.04
3  Italian Restaurant  0.03
4 

In [23]:
# function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]
# code lifted from labs.cognitiveclass.ai/tools

In [24]:
# top venues for each neighborhood
num_top_venues = 5
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns) # create & populate a new dataframe
neighborhoods_venues_sorted['Neighborhood'] = ontario_grouped['Neighborhood']

for ind in np.arange(ontario_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ontario_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant
1,Agincourt,Chinese Restaurant,Lounge,Sandwich Place,Breakfast Spot,Dumpling Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Coffee Shop,Playground,Drugstore,Dim Sum Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Japanese Restaurant,Coffee Shop,Fried Chicken Joint,Video Store
4,"Alderwood, Long Branch",Pizza Place,Gym,Athletics & Sports,Pharmacy,Pool
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Frozen Yogurt Shop,Park,Pharmacy,Pizza Place
6,Bayview Village,Chinese Restaurant,Japanese Restaurant,Café,Bank,Drugstore
7,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Sushi Restaurant,Pharmacy
8,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Cheese Shop,Seafood Restaurant
9,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,Café,General Entertainment,Women's Store


### Step 10 - Visualize Clusters

In [25]:
# cluster the venues geographically
kclusters = 20
ontario_grouped_clustering = ontario_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ontario_grouped_clustering)
kmeans.labels_[0:10]  # check cluster labels generated for each row in the dataframe

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
ontario_merged = geocodedHood
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ontario_merged = ontario_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
ontario_merged.head() # check the last columns!`

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,8.0,Fast Food Restaurant,Women's Store,Drugstore,Dim Sum Restaurant,Diner
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,5.0,Bar,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,6.0,Electronics Store,Medical Center,Pizza Place,Breakfast Spot,Rental Car Location
3,M1G,Scarborough,Woburn,43.770992,-79.216917,19.0,Coffee Shop,Korean Restaurant,Women's Store,Diner,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,6.0,Hakka Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Thai Restaurant


In [26]:
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ontario_merged['Latitude'], ontario_merged['Longitude'], ontario_merged['Neighborhood'], ontario_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color = 'yellow',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Step 11 - Examine Feature-Grouped Clusters 

In [27]:
# Outdoor Rec Hood
ontario_merged.loc[ontario_merged['Cluster Labels'] == 0, ontario_merged.columns[[1] + list(range(5, ontario_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,Scarborough,0.0,Bus Line,Bakery,Soccer Field,Fast Food Restaurant,Bus Station
30,North York,0.0,Park,Other Repair Shop,Airport,Bus Stop,Donut Shop
36,East York,0.0,Park,Beer Store,Skating Rink,Pharmacy,Spa
44,Central Toronto,0.0,Park,Swim School,Dim Sum Restaurant,Bus Line,Empanada Restaurant
64,Central Toronto,0.0,Park,Jewelry Store,Sushi Restaurant,Trail,Empanada Restaurant
72,North York,0.0,Park,Japanese Restaurant,Pub,Bakery,Dessert Shop
79,North York,0.0,Park,Construction & Landscaping,Basketball Court,Bakery,Women's Store


In [32]:
# Active Dog Owners Hood
ontario_merged.loc[ontario_merged['Cluster Labels'] == 1, ontario_merged.columns[[1] + list(range(5, ontario_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
32,North York,1.0,Baseball Field,Food Truck,Women's Store,Dumpling Restaurant,Discount Store
91,Etobicoke,1.0,Baseball Field,Women's Store,Eastern European Restaurant,Discount Store,Dog Run
97,North York,1.0,Baseball Field,Women's Store,Eastern European Restaurant,Discount Store,Dog Run


In [28]:
# Eclectics Hood
ontario_merged.loc[ontario_merged['Cluster Labels'] == 2, ontario_merged.columns[[1] + list(range(5, ontario_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
63,Central Toronto,2.0,Garden,Drugstore,Dim Sum Restaurant,Diner,Discount Store


In [29]:
# Millenial Hood
ontario_merged.loc[ontario_merged['Cluster Labels'] == 3, ontario_merged.columns[[1] + list(range(5, ontario_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,Scarborough,3.0,Pizza Place,Italian Restaurant,Fried Chicken Joint,Pharmacy,Fast Food Restaurant
24,North York,3.0,Coffee Shop,Grocery Store,Pizza Place,Butcher,Pharmacy
34,North York,3.0,Portuguese Restaurant,Coffee Shop,Pizza Place,Hockey Arena,Intersection
35,East York,3.0,Fast Food Restaurant,Pizza Place,Gastropub,Pharmacy,Café
81,York,3.0,Bus Line,Grocery Store,Pizza Place,Convenience Store,Women's Store
89,Etobicoke,3.0,Pizza Place,Gym,Athletics & Sports,Pharmacy,Pool
95,Etobicoke,3.0,Beer Store,Pharmacy,Liquor Store,Pizza Place,Shopping Plaza
99,Etobicoke,3.0,Pizza Place,Middle Eastern Restaurant,Coffee Shop,Sandwich Place,Intersection
100,Etobicoke,3.0,Park,Pizza Place,Mobile Phone Shop,Women's Store,Donut Shop
101,Etobicoke,3.0,Grocery Store,Japanese Restaurant,Coffee Shop,Fried Chicken Joint,Video Store


In [30]:
# Moms Hood
ontario_merged.loc[ontario_merged['Cluster Labels'] == 4, ontario_merged.columns[[1] + list(range(5, ontario_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,Scarborough,4.0,Park,Coffee Shop,Playground,Drugstore,Dim Sum Restaurant
48,Central Toronto,4.0,Park,Playground,Women's Store,Drugstore,Dim Sum Restaurant
50,Downtown Toronto,4.0,Park,Playground,Trail,Empanada Restaurant,Electronics Store


In [36]:
#Global Multicultural Hood
ontario_merged.loc[ontario_merged['Cluster Labels'] == 5, ontario_merged.columns[[1] + list(range(5, ontario_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Scarborough,5.0,Bar,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant
